In [1]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/pbp_related/"
img_DIR_P = "../data/images/players/"
img_DIR_T = "../data/images/teams/"
data_DIR = "../data/pbpdata/"
box_DIR = "../data/box/"
aio_DIR = "../data/all_in_one_metrics/"

source = "pbpstats"

In [2]:
league = "NBA"
season = "2024"
year = season + "-" + str(int(season)+1)[-2:]
with zstd.open(data_DIR + league+"_PBPdata_" + season + ".pkl.zst","rb") as f:
    games_list = dill.load(f)
player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()
len(games_list)

564

In [ ]:
game = games_list[0]

In [7]:
game.game_id

'0022400455'

In [5]:
stats = leaguegamelog.LeagueGameLog(
    player_or_team_abbreviation="T",
    season=season,
    season_type_all_star="Regular Season",
)
df = stats.get_data_frames()[0]

In [18]:
from nba_api.stats.endpoints import teamdetails,commonteamroster

In [32]:
pDB = pd.read_csv("../data/NBA_players_database.csv")

In [34]:
pDB

,pID,Name,From,To
0,76001,Alaa Abdelnaby,1990,1994
1,76002,Zaid Abdul-Aziz,1968,1977
2,76003,Kareem Abdul-Jabbar,1969,1988
3,51,Mahmoud Abdul-Rauf,1990,2000
4,1505,Tariq Abdul-Wahad,1997,2003
...,...,...,...,...
5006,1627826,Ivica Zubac,2016,2024
5007,78650,Matt Zunic,1948,1948
5008,1641783,Tristan da Silva,2024,2024
5009,1628427,Vlatko Čančar,2019,2024


In [53]:
season = 2024
dfa = []
for team in tqdm(teams_list):
    stats = commonteamroster.CommonTeamRoster(team_id=team,season=get_ss(season))
    df1 = stats.get_data_frames()[1]
    dfa.append(df1)
    time.sleep(1)
df2 = pd.concat(dfa)

100%|██████████| 30/30 [00:37<00:00,  1.25s/it]
C:\Users\pansr\AppData\Local\Temp\ipykernel_2036\441056742.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


In [57]:
stats = commonteamroster.CommonTeamRoster(team_id=1610612749,season=get_ss(season))
df1 = stats.get_data_frames()[1]

In [59]:
df1

,TEAM_ID,SEASON,COACH_ID,FIRST_NAME,LAST_NAME,COACH_NAME,IS_ASSISTANT,COACH_TYPE,SORT_SEQUENCE,SUB_SORT_SEQUENCE
0,1610612749,2024,1941,Doc,Rivers,Doc Rivers,1,Head Coach,None,1
1,1610612749,2024,452,Vin,Baker,Vin Baker,2,Assistant Coach,None,5
2,1610612749,2024,1672,Rex,Kalamian,Rex Kalamian,2,Assistant Coach,None,5
3,1610612749,2024,101206,Joe,Prunty,Joe Prunty,2,Assistant Coach,None,5
4,1610612749,2024,201220,David,Joerger,David Joerger,2,Assistant Coach,None,5
5,1610612749,2024,1642209,Pete,Dominguez,Pete Dominguez,2,Assistant Coach,None,5
6,1610612749,2024,1642464,Spencer,Rivers,Spencer Rivers,2,Assistant Coach,None,5
7,1610612749,2024,203811,Greg,Buckner,Greg Buckner,2,Assistant Coach,None,5
8,1610612749,2024,203005,Darvin,Ham,Darvin Ham,2,Assistant Coach,None,5
9,1610612749,2024,1642467,Jack,Herum,Jack Herum,2,Assistant Coach,None,5


In [60]:
df2.columns = map(str.lower,df2.columns)
df3 = df2.rename(columns={"coach_id":"pID"})
df3 = df3[["pID","team_id","coach_type","is_assistant","coach_name"]]
df4 = pd.merge(pDB,df3,left_on="Name",right_on="coach_name")
df4

,pID_x,Name,From,To,pID_y,team_id,coach_type,is_assistant,coach_name
0,201582,Alexis Ajinca,2008,2017,201582,1610612764,Assistant Coach,2,Alexis Ajinca
1,706,Jerome Allen,1995,1996,1627590,1610612765,Assistant Coach,2,Jerome Allen
2,2124,Malik Allen,2001,2010,204229,1610612748,Assistant Coach,2,Malik Allen
3,353,Darrell Armstrong,1994,2007,202599,1610612742,Assistant Coach,2,Darrell Armstrong
4,2486,Maurice Baker,2002,2004,2486,1610612754,Assistant Coach for Player Development,12,Maurice Baker
...,...,...,...,...,...,...,...,...,...
71,2575,Luke Walton,2003,2012,204012,1610612765,Assistant Coach,2,Luke Walton
72,28,Doug West,1989,2000,203242,1610612755,Assistant Coach,2,Doug West
73,2366,Mike Wilks,2002,2009,2366,1610612760,Assistant Coach,2,Mike Wilks
74,78558,Mike Williams,1989,1989,1630737,1610612762,Assistant Coach,2,Mike Williams


In [66]:
df5 = df4.query("is_assistant == 1").sort_values("To").reset_index(drop=True)
df5 = df5.reset_index()
df5.index +=1

In [68]:
df5[["Name","From","To"]]

,Name,From,To
1,Billy Donovan,1987,1987
2,Rick Carlisle,1984,1989
3,Doc Rivers,1983,1995
4,Steve Kerr,1988,2002
5,Doug Christie,1992,2006
6,Tyronn Lue,1998,2008
7,Ime Udoka,2003,2010
8,Jason Kidd,1994,2012
9,Chauncey Billups,1997,2013
10,Willie Green,2003,2014
